### Original: 

ideally, evan would review the material and give us the price projection he is comfortable with (he has been asking us for many pieces of analysis to lead to this question, including molly/surafael/jamies projections and now this one from adam). in the absence of his final 10G and 1G projection, i have recommended using \$ 2500 for 10G and \$ 890 for 1G.

How:
first, please review the logic for my projection. i tried to make it something that would appeal to evan. it is located here: https://docs.google.com/document/d/1R12jdZjNvhG9_4yExXCCIozwX6kZ4Axcs4dXt0y3Efk/edit?disco=AAAADU2fU3g


then calculate, for all state network states, how much would it cost to get the state 1G for all of their districts with <1,000 students, 10G for 1,000-10,000 students? for districts over 10,000 students, however many multiples of 10G get them to their goals? (evan has not requested adjustment for oversubscription, but an alternative we can try if these numbers are low are oversubscription adjustments). note that these projections may change, so it would be wise to make the cost for 1G and 10G easy to malipulate?


### New:

This is a version of state_networks_projections_1G_10G.ipynb that as flat pricing. That is, all districts, regardless if they are meeting or not meeting 1 Mbps goal will get a pricing of $890 for 1G or $2500 for 10G and how this compares to what they are already spending.

### Drive by request:

See details for GA and NE. How were these estimates determined? Add upstream cost, and ia total costs for reference

https://www.pivotaltracker.com/story/show/168150168

In [3]:
import math
import numpy as np
import os
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# get states that are in state network
HOST_DAR = os.environ.get("HOST_DAR")
USER_DAR = os.environ.get("USER_DAR")
PASSWORD_DAR = os.environ.get("PASSWORD_DAR")
DB_DAR = os.environ.get("DB_DAR")
PORT_DAR = os.environ.get("PORT_DAR")
GITHUB = os.environ.get("GITHUB")

In [5]:
#open connection to DB
myConnection = psycopg2.connect( host=HOST_DAR, 
                                user=USER_DAR, 
                                password=PASSWORD_DAR, 
                                database=DB_DAR, 
                                port=PORT_DAR)

### QA Task 1

For all state network states,

- how much would it cost to get the state 1G for all of their districts with <1,000 students, 
- 10G for 1,000-10,000 students?

In [6]:
# sql_query_part1
sql_query = """
select 
  d.district_id, 
  d.funding_year, 
  d.name, 
  d.state_code, 
  d.consortium_affiliation,
  d.consortium_affiliation_ids,
  d.num_students, 
  dbw.ia_bw_mbps_total, 
  dbw.ia_monthly_cost_total, 
  dbw.ia_monthly_cost_per_mbps, 
  dbw.projected_bw_fy2018, 
  dbw.meeting_2018_goal_oversub, 
  dffa.fit_for_ia, 
  dffa.fit_for_ia_cost, 
  ss.sea_name, 
  ss.name as ss_name, 
  ss.state_network 
from 
  ps.districts d 
  JOIN ps.districts_bw_cost dbw on d.funding_year = dbw.funding_year 
  and d.district_id = dbw.district_id 
  join ps.districts_fit_for_analysis dffa ON d.funding_year = dffa.funding_year 
  and d.district_id = dffa.district_id 
  join ps.states_static ss on d.state_code = ss.name 
where 
  d.funding_year = 2019 
  and d.in_universe = true 
  and d.district_type = 'Traditional' 
  and ss.state_network = true
"""

#pull bandwidths from DB
cur = myConnection.cursor()
cur.execute(sql_query)
names = [x[0] for x in cur.description]
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=names)

In [7]:
# convert from decimal to numeric
numeric_cols = ['num_students', 'ia_monthly_cost_total', 'ia_monthly_cost_per_mbps','projected_bw_fy2018']
df[numeric_cols] = df[numeric_cols].astype(float)

In [8]:
df.head()

,district_id,funding_year,name,state_code,consortium_affiliation,consortium_affiliation_ids,num_students,ia_bw_mbps_total,ia_monthly_cost_total,ia_monthly_cost_per_mbps,projected_bw_fy2018,meeting_2018_goal_oversub,fit_for_ia,fit_for_ia_cost,sea_name,ss_name,state_network
0,881421,2019,Marshall County School District,AL,ALABAMA SUPERCOMPUTER AUTHORITY,1005870,5687.0,2000,5284.840000,2.642420,4833.95,False,True,True,ALABAMA DEPARTMENT OF EDUCATION,AL,True
1,881423,2019,Dewitt Public School District,AR,DIS STATE OF ARKANSAS,1006162,1319.0,2000,4260.157791,2.130079,1319.00,True,True,True,ARKANSAS DEPARTMENT OF EDUCATION,AR,True
2,881427,2019,Hoover City School District,AL,ALABAMA SUPERCOMPUTER AUTHORITY,1005870,13938.0,10000,612.500000,0.061250,9756.60,True,True,True,ALABAMA DEPARTMENT OF EDUCATION,AL,True
3,881431,2019,Madison City Schools,AL,ALABAMA SUPERCOMPUTER AUTHORITY,1005870,10440.0,7500,16497.220000,2.199629,8874.00,False,True,True,ALABAMA DEPARTMENT OF EDUCATION,AL,True
4,881435,2019,Warren Public School District 1,AR,DIS STATE OF ARKANSAS,1006162,998.0,2000,3625.866167,1.812933,998.00,True,True,True,ARKANSAS DEPARTMENT OF EDUCATION,AR,True


### Removing specific states with $0k state network spending: NC, AL, MS, CT, SC, WV, HI

In [9]:
df = df[~df.state_code.isin(['NC', 'AL', 'MS', 'CT', 'SC', 'WV', 'HI'])]
df.shape

(3041, 17)

### Masks/Filters

In [10]:
mask_less_than_1k_students = df.num_students <= 1000
mask_1k_to_10k = (df.num_students > 1000) & (df.num_students <= 10000) 
mask_10k_plus = df.num_students > 10000 
mask_not_meeting = df.meeting_2018_goal_oversub == False
mask_meeting = df.meeting_2018_goal_oversub == True
mask_fit_ia = df.fit_for_ia == True
mask_fit_cost = df.fit_for_ia_cost == True

### Add Multiple of 10G's Needed to meet 1 Mbps Goal

In [11]:
# districts with 10k plus students
sub_cols = ['district_id', 'funding_year', 'name', 'state_code','num_students', 'ia_bw_mbps_total', 'projected_bw_fy2018']
df_10k_plus = df[sub_cols][mask_10k_plus]
df_10k_plus

,district_id,funding_year,name,state_code,num_students,ia_bw_mbps_total,projected_bw_fy2018
112,881948,2019,Bentonville Public Schools,AR,16609.0,22000,11626.30
137,882010,2019,Cabot Public School District,AR,10625.0,30000,7437.50
215,882229,2019,Fort Smith Public School,AR,14888.0,20000,10421.60
285,882469,2019,Little Rock School District,AR,23852.0,30000,16696.40
331,882690,2019,Pulaski Co Spec School District,AR,12680.0,24000,8876.00
333,882699,2019,Rogers Public School District 30,AR,15739.0,20000,11017.30
347,882764,2019,Springdale Public Schools,AR,21359.0,30000,14951.30
530,886853,2019,Appoquinimink School District,DE,10525.0,10000,8946.25
534,886859,2019,Christina School District,DE,14538.0,10000,10176.60
537,886862,2019,Indian River School District,DE,10056.0,10000,8547.60


### QA Task 4

In [13]:
# change the price as needed
projected_price_1g = 890
projected_price_10g = 2500

# how many multiples of 10G to get them to their goals? with oversubscription
def round_up_nearest_10k(row):
    return int(math.ceil(row/10000))


# without oversubscription
df_10k_plus['multiple_10G_no_oversub'] = df_10k_plus.num_students.apply(round_up_nearest_10k)

# projected cost to get 10k plus to meet 1 Mbps without oversub by district
df_10k_plus['projected_mrc_10g_10kplus_no_oversub'] = df_10k_plus['multiple_10G_no_oversub']*projected_price_10g

### Filter by State

In [14]:
# filtering for GA and NE
df_ga_ne = df_10k_plus[df_10k_plus.state_code.isin(["GA", "NE"])]

### How much is the state network spending?

### For all state network states, 
- how much is the state network spending? 
- where possible, please calculate this at an application level rather than a district level. this should really just be the upstream line items serving districts in our universe, from the state network applicaiton.

In [15]:
# list of most common consortia ids

# 1006162

# 1004592
# 1008357
# 1009239
# 1014118
# 1015511
# 1020107
# 1016880
# 1020220
# 1021110
# 1032821
# 1037707
# 1035776
# 1055594
# 1051850 | 1051855
# 1049045
# 1047587
# 1047087

### Evan Follow up: Adding the upstream cost by district

In [16]:
# upstream monthly cost total by district

sql_query = """
select 
  dli.district_id,
  d.funding_year,
  ss.state_code, 
  sum(dli.total_monthly_cost) as upstream_monthly_cost_total
  
from 
  ps.districts_line_items dli 
  JOIN ps.districts d on d.district_id = dli.district_id 
  and d.funding_year = dli.funding_year 
  join ps.states_static ss on d.state_code = ss.state_code 
  join ps.line_items li on dli.line_item_id = li.line_item_id 
  join ps.entity_bens_lkp eb on li.applicant_ben = eb.ben 
  and li.funding_year = eb.funding_year 
  
where 
  d.funding_year = 2019 
  and d.district_type = 'Traditional' 
  and d.in_universe = true 
  and ss.state_network = true 
  and dli.purpose in ('upstream') 
  and eb.entity_id in (
    1006162, 1004592, 1008357, 1009239, 
    1014118, 1015511, 1020107, 1020220, 
    1021110, 1032821, 1037707, 1035776, 
    1055594, 1051855, 1047087,
    1051850, 1049045, 1047587
  ) 
group by dli.district_id, d.funding_year, ss.state_code
"""

#pull bandwidths from DB
cur = myConnection.cursor()
cur.execute(sql_query)
names = [x[0] for x in cur.description]
rows = cur.fetchall()
df_sn_spending_district = pd.DataFrame(rows, columns=names)

In [17]:
# filtering for just states "GA" and "NE"
df_upstream_ga_ne = df_sn_spending_district[df_sn_spending_district.state_code.isin(['NE', 'GA'])]

In [20]:
df_upstream_ga_ne

,district_id,funding_year,state_code,upstream_monthly_cost_total
246,889810,2019,GA,1429.2799999999999999857072
247,889835,2019,GA,960.4900000000000000000000
248,889836,2019,GA,2169.200000000000000108460
249,889837,2019,GA,4979.600000000000000000000
250,889838,2019,GA,2169.200000000000000108460
251,889840,2019,GA,2188.6100000000000000000000
252,889857,2019,GA,2169.200000000000000108460
253,889859,2019,GA,1429.2799999999999999857072
254,889860,2019,GA,2169.200000000000000108460
255,889861,2019,GA,2465.1099999999999999506978


In [23]:
df_upstream_ga_ne.district_id.nunique()

272

In [24]:
# filtering original df for just states "GA" and "NE"
df_ga_ne = df[['district_id', 'funding_year', 'name', 'state_code',
               'num_students','ia_bw_mbps_total', 'ia_monthly_cost_total']][df.state_code.isin(['NE', 'GA'])]

### Districts without Upstream Costs

In [29]:
print(f"Number of districts with $0 upstream costs: {len(set(df_ga_ne.district_id.unique()).difference(df_upstream_ga_ne.district_id.unique()))}")

Number of districts with $0 upstream costs: 153


In [31]:
set(df_ga_ne.district_id.unique()).difference(df_upstream_ga_ne.district_id.unique())

{889839,
 889858,
 935899,
 935900,
 935906,
 935907,
 935908,
 935909,
 935911,
 935912,
 935913,
 935914,
 935916,
 935925,
 935926,
 935933,
 935934,
 935936,
 935941,
 935942,
 935943,
 935944,
 935947,
 935948,
 935949,
 935950,
 935952,
 935953,
 935955,
 935956,
 935957,
 935958,
 935961,
 935962,
 935963,
 935965,
 935966,
 935967,
 935968,
 935969,
 935970,
 935971,
 935972,
 935973,
 935974,
 935976,
 935987,
 935991,
 935994,
 935996,
 936001,
 936004,
 936006,
 936007,
 936010,
 936011,
 936015,
 936017,
 936019,
 936023,
 936024,
 936025,
 936026,
 936031,
 936032,
 936033,
 936035,
 936037,
 936038,
 936041,
 936043,
 936044,
 936045,
 936048,
 936050,
 936051,
 936053,
 936054,
 936056,
 936060,
 936063,
 936064,
 936067,
 936068,
 936070,
 936071,
 936072,
 936074,
 936076,
 936077,
 936081,
 936082,
 936084,
 936085,
 936086,
 936089,
 936090,
 936091,
 936093,
 936095,
 936096,
 936097,
 936099,
 936102,
 936105,
 936108,
 936109,
 936110,
 936112,
 936114,
 936117,
 

### Rerun here if needed

In [88]:
# merge both dataframes
df_ga_ne_cost = pd.merge(df_ga_ne, df_upstream_ga_ne[['district_id', 'upstream_monthly_cost_total']], 
                         on='district_id', how='left')

### Add Projected Costs for 1G and 10G

In [91]:
def projected_cost_1G(row):
    if row <= 1000:
        return projected_price_1g
    
def projected_cost_10G(row):
    if (row > 1000) and (row <= 10000):
        return projected_price_10g

# new mrc with $890 1G projected price
df_ga_ne_cost['projected_mrc_1g'] = df_ga_ne_cost.num_students.apply(projected_cost_1G)

# new mrc with $2500 per 10G projected price
df_ga_ne_cost['projected_mrc_10g'] = df_ga_ne_cost.num_students.apply(projected_cost_10G)

### Add Projected Costs for 10G plus

In [95]:
# merge with df_ga_ne_cost
sub_cols = ['district_id', 'multiple_10G_no_oversub',  'projected_mrc_10g_10kplus_no_oversub']
df_result_ga_ne = pd.merge(df_ga_ne_cost, df_10k_plus[sub_cols], how='left', on=['district_id'])

# fill in nans with zeroes
df_result_ga_ne.fillna(0, inplace=True)

### Add Total Projected Cost

In [98]:
# total projected cost is the sum of 1g, 10g and 10gplus
df_result_ga_ne['total_projected_cost'] = df_result_ga_ne['projected_mrc_1g'] + df_result_ga_ne['projected_mrc_10g'] + df_result_ga_ne['projected_mrc_10g_10kplus_no_oversub'] 



In [100]:
# save as a csv
df_result_ga_ne.to_csv("ga_ne_detailed_with_cost.csv", index=False)

In [1]:
# converting to a .py file
!jupyter nbconvert --to script state_networks_projections_1G_10G_flat_pricing_rerun_NE_GA_details.ipynb

[NbConvertApp] Converting notebook state_networks_projections_1G_10G_flat_pricing_rerun_NE_GA_details.ipynb to script
[NbConvertApp] Writing 9306 bytes to state_networks_projections_1G_10G_flat_pricing_rerun_NE_GA_details.py
